In [ ]:
%reload_ext autoreload
%autoreload 2
from pprint import pprint
from IPython.core.display import display, HTML, Markdown
import ipywidgets as widgets
# %run includeme.ipynb # include a notebook from this same directory
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
import boto3
import base64, json
import requests

In [ ]:
region = boto3.session.Session().region_name
account_id = boto3.resource('iam').CurrentUser().arn.split(':')[4]
print(region)
print(account_id)

In [ ]:
"""This will crasssh with an SSL certificate error
client = boto3.client('iot-data') # , region_name=region)
# client = boto3.client('iot') # , region_name=region)
# publish mqtt message
response = client.publish(
    topic='kikkelis/kokkelis',
    qos=1,
    payload=json.dumps({"foo":"bar"})
)
"""

You might need to set the following rights:
```
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": [
                "iot:Publish"
            ],
            "Resource": [
                "*"
            ]
        },
        {
            "Effect": "Allow",
            "Action": [
                "iot:Connect"
            ],
            "Resource": [
                "*"
            ]
        }
    ]
}
```

In [ ]:
# as per https://stackoverflow.com/questions/65285525/ssl-certificate-verify-failed-error-when-publish-mqtt-aws-iot

def get_aws_iot_ats_endpoint():
    """
    Get the "Data-ATS" endpoint instead of the
    untrusted "Symantec" endpoint that's built-in.
    """
    iot_client = boto3.client(
        "iot",
        #aws_access_key_id=AWS_ACCESS_KEY_ID,
        #aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
        #region_name= REGION_NAME,
        verify=True
    )
    details = iot_client.describe_endpoint(endpointType="iot:Data-ATS")
    host = details.get("endpointAddress")
    return f"https://{host}"


IOT_DATA_ENDPOINT = get_aws_iot_ats_endpoint()

client_iot = boto3.client(
    "iot-data",
    #aws_access_key_id=AWS_ACCESS_KEY_ID,
    #aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
    #region_name= REGION_NAME,
    verify=True,
    endpoint_url=IOT_DATA_ENDPOINT
)

In [ ]:
# print(response)

We have created an IoT trigger that sends the data in the MQTT message to a lambda function.

The lambda function then writes the contents of the file into a new file in a bucket.

Just attach this trigger to the lambda function ``SELECT * FROM 'kikkelis/kokkelis'`` => when there is message in channel ``kikkelis/kokkelis``
the payload is passed as-is to the lambda.

In [ ]:
response = client_iot.publish(
    topic='kikkelis/kokkelis',
    qos=1,
    payload=json.dumps({
        "message":"from mqtt",
        "count" : 119911
    })
)